# Configurations

In [ ]:
from pyspark.sql import SparkSession

# Créez la session Spark
spark = SparkSession.builder \
    .appName("PublicTransport") \
    .getOrCreate()

storageaccount = "fidelistorage"
container = "public-transport"
key = "Mea8hwXHm0gWaa3zhrgriV5EjqqECFidwHiUdaN3cqf0wfdP2/SmSEUaxCrNJuApXFIqQno2zJjG+AStx50KwQ=="


spark.conf.set(
    f"fs.azure.account.key.{storageaccount}.dfs.core.windows.net", 
    key
)

# Spécifiez le chemin du répertoire raw dans Azure Data Lake Storage Gen2
raw_data_path = "abfss://public-transport@fidelistorage.dfs.core.windows.net/raw/"
processed_path = "abfss://public-transport@fidelistorage.dfs.core.windows.net/processed/"
archives_path = "abfss://public-transport@fidelistorage.dfs.core.windows.net/archives/"


# Archivage des données brutes (raw data)

In [ ]:
from datetime import datetime
import os
import shutil

# Récupérez la liste des fichiers dans le dossier raw/
raw_files = dbutils.fs.ls(raw_data_path)

# Obtenez la date actuelle pour créer un dossier d'archivage avec cette date
current_date = datetime.now().strftime("%Y-%m-%d")
archive_folder = os.path.join(archives_path, "raw", current_date)

# Créez le dossier d'archivage pour les données brutes
dbutils.fs.mkdirs(archive_folder)

# Déplacez chaque fichier du dossier raw/ vers le dossier d'archivage pour les données brutes
for file in raw_files:
    file_name = os.path.basename(file.path)
    archive_file_path = os.path.join(archive_folder, file_name)
    dbutils.fs.mv(file.path, archive_file_path)


# Archivage des données traitées

In [ ]:
# Récupérez la liste des fichiers dans le dossier processed/
processed_files = dbutils.fs.ls(processed_path)

# Obtenez la date actuelle pour créer un dossier d'archivage avec cette date
current_date = datetime.now().strftime("%Y-%m-%d")
archive_folder = os.path.join(archives_path, "processed", current_date)

# Créez le dossier d'archivage pour les données transformées
dbutils.fs.mkdirs(archive_folder)

# Déplacez chaque fichier du dossier processed/ vers le dossier d'archivage pour les données transformées
for file in processed_files:
    file_name = os.path.basename(file.path)
    archive_file_path = os.path.join(archive_folder, file_name)
    dbutils.fs.mv(file.path, archive_file_path, recurse=True)

# Vous pouvez également supprimer les fichiers du dossier processed/ si nécessaire
# for file in processed_files:
#     dbutils.fs.rm(file.path)

# Fermez la session Spark
# spark.stop()
